# Agent Chat with Multi-Modality Models

We use LLaVA as an example for the multi-modality feature. 


This notebook contains the following information and examples:

1. Setup LLaVA 
    - Option 1: Use API calls from `Replicate`
    - Option 2: Setup LLaVA locally (requires GPU)
2. Application 1: Garden helper
3. Application 2: Image Regeneration

In [13]:
# We use this variable to control where you want to host LLaVA, locally or remotely?
# More details in the two setup options below.

LLAVA_MODE = "local" # Either "local" or "remote"

assert LLAVA_MODE in ["local", "remote"]

# (Option 1, preferred) Use API Calls from Replicate [Remote]
We can also use [Replicate](https://replicate.com/yorickvp/llava-13b/api) to use LLaVA directly, which will host the model for you.

1. Run `pip install replicate` to install the package
2. You need to get an API key from Replicate from your [account setting page](https://replicate.com/account/api-tokens)
3. Next, copy your API token and authenticate by setting it as an environment variable:
    `export REPLICATE_API_TOKEN=<paste-your-token-here>` 
4. You need to enter your credit card information for Replicate 🥲
    

In [3]:
# pip install replicate
# import os
## alternatively, you can put your API key here for the environment variable.
# os.environ["REPLICATE_API_TOKEN"] = "r8_xyz your api key goes here~"

In [4]:
import replicate


In [5]:
# img = get_image_data("https://github.com/haotian-liu/LLaVA/raw/main/images/llava_logo.png", use_b64=True)
# img = 'data:image/jpeg;base64,' + img

# response = replicate.run(
#     "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
#     input={"image": img,
#            "prompt": "Describe the in poetry."}
# )
# output = ""
# for item in response:
#     output += item
# print(output)

## [Option 2] Setup LLaVA Locally
Please follow the LLaVA GitHub [page](https://github.com/haotian-liu/LLaVA/) to install LLaVA, download the weights, and start the server.

For instance, here are some important steps:
```bash
# Download the package
git clone https://github.com/haotian-liu/LLaVA.git
cd LLaVA

# Install the inference package
conda create -n llava python=3.10 -y
conda activate llava
pip install --upgrade pip  # enable PEP 660 support
pip install -e .

# Download and serve the model
python -m llava.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path liuhaotian/llava-v1.5-7b
```

Some helpful packages and dependencies:
```bash
conda install -c nvidia cuda-toolkit
```


### Launch

In one terminal, start the controller first:
```bash
python -m llava.serve.controller --host 0.0.0.0 --port 10000
```


Then, in another terminal, start the worker, which will load the model to the GPU:
```bash
python -m llava.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path liuhaotian/llava-v1.5-13b
``

**Note: make sure the environment of this notebook also installed the llava package from `pip install -e .`**

In [2]:
# Run this code block only if you want to run LlaVA locally
import requests
import json
from llava.conversation import default_conversation as conv
from llava.conversation import Conversation

# Setup some global constants for convenience
# Note: make sure the addresses below are consistent with your setup in LLaVA 
WORKER_ADDR = "http://0.0.0.0:40000"
CONTROLLER_ADDR = "http://0.0.0.0:10000"
SEP =  conv.sep
ret = requests.post(CONTROLLER_ADDR + "/list_models")
print(ret.json())
MODEL_NAME = ret.json()["models"][0]
print("Model Name:", MODEL_NAME)

[2023-10-17 16:11:27,800] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
{'models': ['llava-v1.5-13b']}
Model Name: llava-v1.5-13b


# Multi-Modal Functions

In [6]:
import base64
import re
from io import BytesIO

from PIL import Image


def extract_img_paths(paragraph: str) -> list:
    """
    Extract image paths (URLs or local paths) from a text paragraph.
    
    Parameters:
        paragraph (str): The input text paragraph.
        
    Returns:
        list: A list of extracted image paths.
    """
    # Regular expression to match image URLs and file paths
    img_path_pattern = re.compile(r'\b(?:http[s]?://\S+\.(?:jpg|jpeg|png|gif|bmp)|\S+\.(?:jpg|jpeg|png|gif|bmp))\b', 
                                  re.IGNORECASE)
    
    # Find all matches in the paragraph
    img_paths = re.findall(img_path_pattern, paragraph)
    return img_paths


def get_image_data(image_file, use_b64=True):
    if image_file.startswith('http://') or image_file.startswith('https://'):
        response = requests.get(image_file)
        content = response.content
    elif image_file.startswith("data:image/png;base64,"):
        return image_file.replace("data:image/png;base64,", "")
    else:
        image = Image.open(image_file).convert('RGB')
        content = open(image_file, "rb").read()
        
    if use_b64:
        return base64.b64encode(content).decode('utf-8')
    else:
        return content
    
def _to_pil(data):
    return Image.open(BytesIO(base64.b64decode(data)))


def llava_call(prompt:str, model_name: str=MODEL_NAME, images: list=[], max_new_tokens:int=1000) -> str:
    """
    Makes a call to the LLaVA service to generate text based on a given prompt and optionally provided images.

    Args:
        - prompt (str): The input text for the model. Any image paths or placeholders in the text should be replaced with "<image>".
        - model_name (str, optional): The name of the model to use for the text generation. Defaults to the global constant MODEL_NAME.
        - images (list, optional): A list of image paths or URLs. If not provided, they will be extracted from the prompt.
            If provided, they will be appended to the prompt with the "<image>" placeholder.
        - max_new_tokens (int, optional): Maximum number of new tokens to generate. Defaults to 1000.

    Returns:
        - str: Generated text from the model.

    Raises:
        - AssertionError: If the number of "<image>" tokens in the prompt and the number of provided images do not match.
        - RunTimeError: If any of the provided images is empty.

    Notes:
    - The function uses global constants: WORKER_ADDR and SEP.
    - Any image paths or URLs in the prompt are automatically replaced with the "<image>" token.
    - If more images are provided than there are "<image>" tokens in the prompt, the extra tokens are appended to the end of the prompt.
    """
    if len(images) == 0:
        images = extract_img_paths(prompt)
        for im in images:
            prompt = prompt.replace(im, "<image>")
    else:
        # Append the <image> token if missing
        assert prompt.count("<image>") <= len(images), "the number "
        "of image token in prompt and in the images list should be the same!"
        num_token_missing = len(images) - prompt.count("<image>")
        prompt += " <image> " * num_token_missing

    
    images = [get_image_data(x) for x in images]
    
    for im in images:
        if len(im) == 0:
            raise RunTimeError("An image is empty!")
            
    headers = {"User-Agent": "LLaVA Client"}
    pload = {
        "model": model_name,
        "prompt": prompt,
        "max_new_tokens": max_new_tokens,
        "temperature": 0.5,
        "stop": SEP,
        "images": images,
    }

    if LLAVA_MODE == "local":
        response = requests.post(WORKER_ADDR + "/worker_generate_stream", headers=headers,
                json=pload, stream=False)

        for chunk in response.iter_lines(chunk_size=8192, decode_unicode=False, delimiter=b"\0"):
            if chunk:
                data = json.loads(chunk.decode("utf-8"))
                output = data["text"].split(SEP)[-1]
    elif LLAVA_MODE == "remote":
        # The Replicate version of the model only support 1 image for now.
        img = 'data:image/jpeg;base64,' + images[0]
        response = replicate.run(
            "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
            input={"image": img, "prompt": prompt.replace("<image>", " ")}
        )
        # The yorickvp/llava-13b model can stream output as it's running.
        # The predict method returns an iterator, and you can iterate over that output.
        output = ""
        for item in response:
            # https://replicate.com/yorickvp/llava-13b/versions/2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591/api#output-schema
            output += item
        
    # Remove the prompt and the space.
    output = output.replace(prompt, "").strip().rstrip()
    
    return output


Here is the image that we are going to use.

![Image](https://github.com/haotian-liu/LLaVA/raw/main/images/llava_logo.png)

In [7]:
out = llava_call("Describe this image: <image>", 
                 images=["https://github.com/haotian-liu/LLaVA/raw/main/images/llava_logo.png"])
print(out)

The image features a small, red, and black toy with glasses. The toy appears to be a stuffed animal or a figurine, possibly a cute red lizard or a dragon. The red color and the flames on the toy give it a fiery appearance. The toy is placed on a grey surface, which might be a table or a shelf. The glasses add a playful and quirky touch to the overall design, making the toy appear more charming and unique.


In [8]:
out = llava_call("Describe this image in one sentence: https://github.com/haotian-liu/LLaVA/raw/main/images/llava_logo.png")
print(out)

The statue is a small animal, possibly a horse, with a fire element in its design. It has flames on its back and is wearing glasses. The fire element adds a unique and interesting visual effect to the statue, making it an eye-catching piece of art.


## Application 1: Garden Helper


Here we demonstrate a very simple multi-agent collaboration on creating visualization.

The user will upload an image of their garden, the image agent (with LLaVA backend) will read the image and describe the problem. Then, the suggestion agent (AssistantAgent with GPT model) will give suggestions on how to treat the problem.


Here, we found a problem in our garden and took a photo:
![](http://th.bing.com/th/id/R.105d684e5df7d540e61f6300d0bd374e?rik=PR8LCyvpe93DZA&pid=ImgRaw&r=0)

In [10]:
import autogen
from autogen import AssistantAgent, Agent

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

llm_config = {"config_list": config_list_gpt4, "seed": 42}

In [15]:
class ImageAgent(AssistantAgent):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.register_reply([Agent, None], reply_func=ImageAgent._image_reply)
        
    def _image_reply(
        self,
        messages=None,
        sender=None, config=None
    ):
        # Note: we did not use "llm_config" yet.
        # TODO: make the LLaVA design compatible with llm_config
        if all((messages is None, sender is None)):
            error_msg = f"Either {messages=} or {sender=} must be provided."
            logger.error(error_msg)
            raise AssertionError(error_msg)

        if messages is None:
            messages = self._oai_messages[sender]

        msg = messages[-1]["content"]
#         prompt = "For the image: <image>\n\n" + self.system_message

        prompt = f"[INST]{self.system_message}[/INST]" + msg
        
        out = ""
        retry = 5
        while len(out) == 0 and retry > 0:
            # image names will be inferred automatically from llava_call
            out = llava_call(prompt=prompt)
            retry -= 1
            
        assert out != "", "Empty response from LLaVA."
        
        
        return True, out


image_agent = ImageAgent(
    name="image-explainer",
    system_message="You are a garden helper! You need to describe what are in the image, and highlight the problems with the plants and the garden"
)

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "groupchat"
    },
    human_input_mode="NEVER",
    llm_config=llm_config,
)
suggestion_giver = autogen.AssistantAgent(
    name=
    "Suggestion-Giver",
    system_message="Give me treatment suggestions for my garden! You can find the description of my image from the image-explainer agent. Keep the answer concise and short.",
    llm_config=llm_config,
)


groupchat = autogen.GroupChat(agents=[user_proxy, image_agent, suggestion_giver],
                              messages=[],
                              max_round=3)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


# Ask the question with an image
user_proxy.initiate_chat(manager, 
                         message="What's wrong with my strawberries? http://th.bing.com/th/id/R.105d684e5df7d540e61f6300d0bd374e?rik=PR8LCyvpe93DZA&pid=ImgRaw&r=0")


User_proxy (to chat_manager):

What's wrong with my strawberries? http://th.bing.com/th/id/R.105d684e5df7d540e61f6300d0bd374e?rik=PR8LCyvpe93DZA&pid=ImgRaw&r=0

--------------------------------------------------------------------------------
image-explainer (to chat_manager):

I am a garden helper and I have been asked to look at this photo of strawberries. The plants appear to be in poor health, with wilted leaves and small, unripe berries. There are also some pests visible on the plants, which could be contributing to their poor health. It is possible that the plants are not receiving enough water, or there may be issues with the soil quality. Additionally, the presence of pests could indicate that the plants are not being adequately maintained. To improve the health of these plants, it would be beneficial to address these issues by providing proper care, such as ensuring that the plants receive sufficient water and nutrients, and implementing measures to control pests.

------------

# Application 2: Image Regeneration


> Ackowledgement: We draw inspirations from the [GitHub project](https://github.com/JayZeeDesign/vision-agent-with-llava/blob/main/app.py) and [YouTube video](https://www.youtube.com/watch?v=JgVb8A6OJwM&t=1s&ab_channel=AIJason) from [AI Jason](https://www.ai-jason.com/)

Input an image, and two agents collaborate together:
- Generator: an agent that generate better prompts to plot figure, and invoke DALLE to generate images. 
- Critic: a discriminator to tell why the two images are different
